In [2]:
import os, sys
print("python:", sys.executable)
print("FRED_API_KEY set:", os.environ.get("FRED_API_KEY") is not None)

python: c:\Users\jonat\anaconda3\envs\investsys\python.exe
FRED_API_KEY set: True


In [4]:
%pip install -U fredapi yfinance plotly pandas numpy sqlalchemy psycopg2-binary

   ---------------------------------------- 0.0/12.9 MB ? eta -:--:--
   ---------------------------------------- 12.9/12.9 MB 203.9 MB/s eta 0:00:00

  Attempting uninstall: numpy

    Found existing installation: numpy 1.24.3

   ---------------------------------------- 0/2 [numpy]
    Uninstalling numpy-1.24.3:
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
      Successfully uninstalled numpy-1.24.3
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------------------------- 0/2 [numpy]
   ---------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.1.0 requires numpy<2,>=1, but you have numpy 2.2.6 which is incompatible.
langchain-community 0.0.10 requires numpy<2,>=1, but you have numpy 2.2.6 which is incompatible.
langchain-openai 0.0.5 requires numpy<2,>=1, but you have numpy 2.2.6 which is incompatible.
scikit-learn 1.3.2 requires numpy<2.0,>=1.17.3, but you have numpy 2.2.6 which is incompatible.
scipy 1.11.4 requires numpy<1.28.0,>=1.21.6, but you have numpy 2.2.6 which is incompatible.


In [5]:
import os
import numpy as np
import pandas as pd

from fredapi import Fred
import yfinance as yf

import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px

def set_plotly_renderer():
    """
    VS Code notebooks usually work with 'vscode'.
    If you still see blank output, switch to 'browser'.
    """
    for r in ["vscode", "notebook_connected", "notebook", "browser"]:
        try:
            pio.renderers.default = r
            return r
        except Exception:
            continue
    return None

renderer = set_plotly_renderer()
print("Plotly renderer:", renderer)

Plotly renderer: vscode


In [6]:
START = "2000-01-01"

FRED_SERIES = {
    # Growth / activity
    "INDPRO":   "Industrial Production",
    "PAYEMS":   "Nonfarm Payrolls",
    "UNRATE":   "Unemployment Rate",
    "ICSA":     "Initial Claims",
    "RSAFS":    "Retail Sales",
    "GACDISA066MSFRBNY":  "NY Fed Empire: General Business Conditions (diffusion)",
    "GACDFSA066MSFRBPHI": "Philly Fed: Current General Activity (diffusion)",

    # Inflation
    "CPILFESL": "Core CPI",
    "PCEPILFE": "Core PCE",
    "CES0500000003": "Avg Hourly Earnings",

    # Liquidity / financial conditions / credit
    "WALCL":    "Fed Balance Sheet",
    "RRPONTSYD":"ON RRP",
    "WTREGEN":  "Treasury General Account (TGA)",
    "NFCI":     "Chicago Fed NFCI",
    "BAMLH0A0HYM2": "HY OAS",
}

BUCKETS = {
    "growth":   ["INDPRO","PAYEMS","UNRATE","ICSA","RSAFS","GACDISA066MSFRBNY","GACDFSA066MSFRBPHI"],
    "inflation":["CPILFESL","PCEPILFE","CES0500000003"],
    "liquidity":["WALCL","RRPONTSYD","WTREGEN","NFCI","BAMLH0A0HYM2"],
}

# +1 means "higher value = more of that bucket"
# -1 means invert because higher value = worse growth or tighter liquidity
SIGN = {
    "INDPRO": +1,
    "PAYEMS": +1,
    "UNRATE": -1,           # higher unemployment = worse growth
    "ICSA":   -1,           # higher claims = worse growth
    "RSAFS":  +1,
    "GACDISA066MSFRBNY": +1,
    "GACDFSA066MSFRBPHI": +1,

    "CPILFESL": +1,
    "PCEPILFE": +1,
    "CES0500000003": +1,

    "WALCL": +1,            # bigger balance sheet ~ easier liquidity
    "RRPONTSYD": -1,        # more RRP usage = drains liquidity
    "WTREGEN":  -1,         # higher TGA = drains liquidity
    "NFCI":     -1,         # higher NFCI = tighter conditions
    "BAMLH0A0HYM2": -1,     # wider spreads = tighter
}

ASSETS = ["SPY","TLT","GLD","UUP","HYG"]

In [7]:
key = os.environ.get("FRED_API_KEY")
print("FRED_API_KEY set:", key is not None)
if key is None:
    raise ValueError("FRED_API_KEY is not set in this kernel. Restart VS Code after setting it in Windows env vars.")

FRED_API_KEY set: True


In [8]:
fred = Fred(api_key=os.environ["FRED_API_KEY"])

def fetch_fred_series(series_dict, start="2000-01-01"):
    out = []
    failed = {}

    for code, name in series_dict.items():
        try:
            s = fred.get_series(code)
            s.index = pd.to_datetime(s.index)
            s = s.loc[start:]
            out.append(s.rename(code))
        except Exception as e:
            failed[code] = str(e)

    raw = pd.concat(out, axis=1).sort_index()
    raw.index.name = "DATE"

    if failed:
        print("⚠️ Failed series:")
        for k,v in failed.items():
            print(" -", k, "=>", v[:120], "...")
    else:
        print("All series fetched successfully.")

    return raw

raw = fetch_fred_series(FRED_SERIES, START)
raw.tail()

All series fetched successfully.


,INDPRO,PAYEMS,UNRATE,ICSA,RSAFS,GACDISA066MSFRBNY,GACDFSA066MSFRBPHI,CPILFESL,PCEPILFE,CES0500000003,WALCL,RRPONTSYD,WTREGEN,NFCI,BAMLH0A0HYM2
DATE,,,,,,,,,,,,,,,
2025-12-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.893,NaN,NaN,2.83
2025-12-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6581231.0,4.803,837120.0,NaN,2.84
2025-12-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2025-12-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,20.339,NaN,NaN,2.86
2025-12-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.551,NaN,NaN,NaN


In [9]:
# Daily grid + ffill (weekend-safe)
raw_d = raw.asfreq("D").ffill()

# pre-history cleanup: only fill remaining NaNs for RRP (facility didn't exist historically)
raw_d["RRPONTSYD"] = raw_d["RRPONTSYD"].fillna(0)

# Month-end snapshot
m = raw_d.resample("ME").last()

print("Monthly rows:", len(m), "cols:", m.shape[1])
m.tail()

Monthly rows: 312 cols: 15


,INDPRO,PAYEMS,UNRATE,ICSA,RSAFS,GACDISA066MSFRBNY,GACDFSA066MSFRBPHI,CPILFESL,PCEPILFE,CES0500000003,WALCL,RRPONTSYD,WTREGEN,NFCI,BAMLH0A0HYM2
DATE,,,,,,,,,,,,,,,
2025-08-31,101.6019,159485.0,4.3,236000.0,731700.0,11.9,-0.3,329.793,126.707,36.58,6603384.0,77.898,589998.0,-0.52904,2.84
2025-09-30,101.6729,159593.0,4.4,224000.0,732444.0,-8.7,23.2,330.542,126.955,36.65,6608395.0,49.071,804856.0,-0.52770,2.80
2025-10-31,101.6160,159488.0,4.4,220000.0,732633.0,10.7,-12.8,330.542,126.955,36.81,6587034.0,51.802,957990.0,-0.51640,2.94
2025-11-30,101.7935,159552.0,4.6,192000.0,732633.0,18.7,-1.7,331.068,126.955,36.86,6552419.0,7.561,903394.0,-0.52938,2.92
2025-12-31,101.7935,159552.0,4.6,214000.0,732633.0,-3.9,-10.2,331.068,126.955,36.86,6581231.0,10.551,837120.0,-0.54911,2.86


In [10]:
(m.isna().mean()*100).sort_values(ascending=False)

CES0500000003         23.717949
WALCL                 11.217949
WTREGEN               11.217949
GACDISA066MSFRBNY      5.769231
INDPRO                 0.000000
PAYEMS                 0.000000
UNRATE                 0.000000
ICSA                   0.000000
RSAFS                  0.000000
GACDFSA066MSFRBPHI     0.000000
CPILFESL               0.000000
PCEPILFE               0.000000
RRPONTSYD              0.000000
NFCI                   0.000000
BAMLH0A0HYM2           0.000000
dtype: float64

In [11]:
def fetch_assets_monthly(tickers, start="2000-01-01"):
    px_daily = yf.download(tickers, start=start, auto_adjust=True, progress=False)["Close"]
    if isinstance(px_daily, pd.Series):
        px_daily = px_daily.to_frame()

    px_m = px_daily.resample("ME").last()
    rets_m = px_m.pct_change()

    px_m.index.name = "DATE"
    rets_m.index.name = "DATE"
    return px_m, rets_m

asset_px_m, asset_rets_m = fetch_assets_monthly(ASSETS, START)
asset_rets_m.tail()

Ticker,GLD,HYG,SPY,TLT,UUP
DATE,,,,,
2025-08-31,0.049875,0.011053,0.020520,0.000127,-0.018996
2025-09-30,0.117584,0.008860,0.035620,0.035910,0.005115
2025-10-31,0.035587,-0.000111,0.023837,0.013811,0.025445
2025-11-30,0.053678,0.007464,0.001950,0.002723,0.000000
2025-12-31,0.027637,0.004926,0.009500,-0.016423,-0.010938


In [12]:
def safe_log(x):
    x = x.copy()
    x = x.where(x > 0)
    return np.log(x)

def build_features(m):
    f = pd.DataFrame(index=m.index)

    yoy_pct = ["INDPRO","PAYEMS","RSAFS","CPILFESL","PCEPILFE","CES0500000003","ICSA"]
    for c in yoy_pct:
        if c in m.columns:
            f[c] = m[c].pct_change(12) * 100

    if "UNRATE" in m.columns:
        f["UNRATE"] = m["UNRATE"] - m["UNRATE"].shift(12)

    # Diffusion surveys as level
    for c in ["GACDISA066MSFRBNY","GACDFSA066MSFRBPHI"]:
        if c in m.columns:
            f[c] = m[c]

    # Liquidity: 3m change in log levels (stabilises scale)
    for c in ["WALCL","RRPONTSYD","WTREGEN"]:
        if c in m.columns:
            f[c] = safe_log(m[c]).diff(3)

    # Financial conditions: levels
    for c in ["NFCI","BAMLH0A0HYM2"]:
        if c in m.columns:
            f[c] = m[c]

    return f

feat = build_features(m)
feat.tail()

,INDPRO,PAYEMS,RSAFS,CPILFESL,PCEPILFE,CES0500000003,ICSA,UNRATE,GACDISA066MSFRBNY,GACDFSA066MSFRBPHI,WALCL,RRPONTSYD,WTREGEN,NFCI,BAMLH0A0HYM2
DATE,,,,,,,,,,,,,,,
2025-08-31,1.165976,0.892620,4.972606,3.112191,2.906731,3.831961,3.508772,0.1,11.9,-0.3,-0.010524,-1.399256,0.233353,-0.52904,2.84
2025-09-30,1.868079,0.807888,4.180185,3.025543,2.825879,3.736202,-1.321586,0.3,-8.7,23.2,-0.008109,-2.239546,0.792480,-0.52770,2.80
2025-10-31,2.157948,0.713573,3.469007,2.752356,2.521965,3.748591,0.917431,0.3,10.7,-12.8,-0.008397,-1.420625,0.949965,-0.51640,2.94
2025-11-30,2.518821,0.588202,2.876942,2.618879,2.414450,3.510250,-14.666667,0.4,18.7,-1.7,-0.007748,-2.332397,0.426040,-0.52938,2.92
2025-12-31,1.461417,0.383788,2.086080,2.403989,2.221489,3.307175,2.392344,0.5,-3.9,-10.2,-0.004119,-1.537048,0.039304,-0.54911,2.86


In [13]:
def rolling_z(df, window=60, min_periods=24):
    mu = df.rolling(window, min_periods=min_periods).mean()
    sd = df.rolling(window, min_periods=min_periods).std()
    return (df - mu) / sd

Z_WINDOW = 60  # 5y window; change to 36 if you want it to “start” earlier
z = rolling_z(feat, window=Z_WINDOW, min_periods=24)

# sign-align
for c in z.columns:
    if c in SIGN:
        z[c] = z[c] * SIGN[c]

z.tail()

,INDPRO,PAYEMS,RSAFS,CPILFESL,PCEPILFE,CES0500000003,ICSA,UNRATE,GACDISA066MSFRBNY,GACDFSA066MSFRBPHI,WALCL,RRPONTSYD,WTREGEN,NFCI,BAMLH0A0HYM2
DATE,,,,,,,,,,,,,,,
2025-08-31,0.064646,-0.301597,-0.343299,-0.669411,-0.624527,-0.761621,0.026831,-0.286693,0.714945,-0.368649,-0.247939,0.951982,-0.469575,0.693640,1.203018
2025-09-30,0.220487,-0.379863,-0.434767,-0.756542,-0.725545,-0.850563,0.033062,-0.418309,-0.558469,0.870062,-0.168839,1.363006,-1.301223,0.679970,1.233601
2025-10-31,0.273859,-0.466573,-0.515963,-0.975432,-1.017067,-0.813333,-0.052923,-0.452525,0.659419,-1.001699,-0.160012,1.026001,-1.487391,0.607370,1.024442
2025-11-30,0.347852,-0.574797,-0.585166,-1.098145,-1.146903,-1.063352,0.116101,-0.532684,1.129141,-0.388664,-0.119410,1.498962,-0.683874,0.695147,1.017788
2025-12-31,0.026933,-0.732149,-0.674523,-1.279813,-1.348655,-1.257033,-0.221065,-0.616540,-0.264472,-0.822044,0.023771,1.023060,-0.101933,0.830167,1.068276


In [14]:
scores = pd.DataFrame(index=z.index)
for bucket, cols in BUCKETS.items():
    cols_ok = [c for c in cols if c in z.columns]
    scores[bucket] = z[cols_ok].mean(axis=1)

scores.tail()

,growth,inflation,liquidity
DATE,,,
2025-08-31,-0.070545,-0.685187,0.426225
2025-09-30,-0.095400,-0.777550,0.361303
2025-10-31,-0.222343,-0.935277,0.202082
2025-11-30,-0.069745,-1.102800,0.481723
2025-12-31,-0.471980,-1.295167,0.568668


In [15]:
def classify_regime(g, i):
    if pd.isna(g) or pd.isna(i):
        return np.nan
    if g >= 0 and i >= 0:
        return "Reflation (G↑ I↑)"
    if g < 0 and i >= 0:
        return "Stagflation (G↓ I↑)"
    if g < 0 and i < 0:
        return "Deflation (G↓ I↓)"
    return "Goldilocks (G↑ I↓)"

regime = scores.apply(lambda r: classify_regime(r["growth"], r["inflation"]), axis=1)
regime.value_counts(dropna=True)

Reflation (G↑ I↑)      95
Stagflation (G↓ I↑)    76
Goldilocks (G↑ I↓)     57
Deflation (G↓ I↓)      49
Name: count, dtype: int64

In [16]:
df_reg = pd.DataFrame({
    "regime": regime,
    "g": scores["growth"],
    "i": scores["inflation"],
    "l": scores["liquidity"]
})
df_reg[df_reg["regime"].str.contains("Deflation", na=False)].tail(24)

,regime,g,i,l
DATE,,,,
2020-11-30,Deflation (G↓ I↓),-0.447220,-0.198590,0.267096
2021-02-28,Deflation (G↓ I↓),-0.270185,-0.043073,-0.193184
2023-11-30,Deflation (G↓ I↓),-0.131603,-0.013652,-0.179235
2023-12-31,Deflation (G↓ I↓),-0.177051,-0.084813,0.042462
2024-01-31,Deflation (G↓ I↓),-0.436832,-0.040226,0.067991
2024-02-29,Deflation (G↓ I↓),-0.124392,-0.132277,0.144284
2024-03-31,Deflation (G↓ I↓),-0.112415,-0.121412,0.168799
2024-04-30,Deflation (G↓ I↓),-0.201454,-0.255921,0.115254
2024-05-31,Deflation (G↓ I↓),-0.103051,-0.332502,0.155223


In [17]:
fig = go.Figure()
for c in ["growth", "inflation", "liquidity"]:
    fig.add_trace(go.Scatter(x=scores.index, y=scores[c], mode="lines", name=f"{c.title()} (z)"))

fig.add_hline(y=0)
fig.update_layout(
    title="Macro Tape — Growth / Inflation / Liquidity (rolling z composites)",
    hovermode="x unified",
    height=420
)
fig.show()

In [18]:
df_scatter = scores.copy()
df_scatter["regime"] = regime
df_scatter = df_scatter.dropna(subset=["growth","inflation","regime"]).reset_index().rename(columns={"DATE":"date"})

fig = px.scatter(
    df_scatter,
    x="growth", y="inflation",
    color="regime",
    hover_data=["date"],
    title="Growth vs Inflation — Regime Map"
)
fig.add_hline(y=0); fig.add_vline(x=0)
fig.update_layout(height=450)
fig.show()

In [19]:
rets = asset_rets_m.copy()
rets.columns = [f"1m_{c}" for c in rets.columns]

df_r = pd.concat([regime.rename("regime"), rets], axis=1).dropna()
heat = df_r.groupby("regime")[rets.columns].mean() * 100  # %

fig = px.imshow(
    heat,
    text_auto=".2f",
    aspect="auto",
    title="Avg 1M Returns by Regime (%)"
)
fig.update_layout(height=420)
fig.show()

In [ ]:
latest_dt = z.dropna(how="all").index.max()
latest = z.loc[latest_dt].dropna().sort_values()

k = 6
neg = latest.head(k)
pos = latest.tail(k)

drivers = pd.concat([neg, pos]).to_frame("z").reset_index().rename(columns={"index":"code"})
drivers["name"] = drivers["code"].map(FRED_SERIES).fillna(drivers["code"])

fig = px.bar(
    drivers.sort_values("z"),
    x="z", y="name",
    orientation="h",
    title=f"Top Drivers (z-scores) — as of {latest_dt.date()}"
)

fig.update_layout(height=500, yaxis_title="")
fig.show()

In [21]:
from sqlalchemy import create_engine

# EDIT THESE if your docker-compose uses different creds
PG_USER = "postgres"
PG_PASS = "postgres"   # change if your compose sets another password
PG_HOST = "localhost"
PG_PORT = 5432
PG_DB   = "macro"

engine = create_engine(f"postgresql+psycopg2://{PG_USER}:{PG_PASS}@{PG_HOST}:{PG_PORT}/{PG_DB}")

# Save: raw monthly snapshot, features, z, scores, regime, asset returns
m_out = m.copy(); m_out.index.name = "date"
feat_out = feat.copy(); feat_out.index.name = "date"
z_out = z.copy(); z_out.index.name = "date"
scores_out = scores.copy(); scores_out.index.name = "date"
reg_out = regime.rename("regime").to_frame(); reg_out.index.name = "date"
rets_out = asset_rets_m.copy(); rets_out.index.name = "date"

m_out.to_sql("macro_monthly_raw", engine, if_exists="replace", index=True)
feat_out.to_sql("macro_monthly_feat", engine, if_exists="replace", index=True)
z_out.to_sql("macro_monthly_z", engine, if_exists="replace", index=True)
scores_out.to_sql("macro_monthly_scores", engine, if_exists="replace", index=True)
reg_out.to_sql("macro_monthly_regime", engine, if_exists="replace", index=True)
rets_out.to_sql("asset_monthly_rets", engine, if_exists="replace", index=True)

print("✅ Wrote tables: macro_monthly_raw, macro_monthly_feat, macro_monthly_z, macro_monthly_scores, macro_monthly_regime, asset_monthly_rets")

✅ Wrote tables: macro_monthly_raw, macro_monthly_feat, macro_monthly_z, macro_monthly_scores, macro_monthly_regime, asset_monthly_rets


In [22]:
import pandas as pd
pd.read_sql("SELECT * FROM macro_monthly_scores ORDER BY date DESC LIMIT 5", engine)

,date,growth,inflation,liquidity
0,2025-12-31,-0.471980,-1.295167,0.568668
1,2025-11-30,-0.069745,-1.102800,0.481723
2,2025-10-31,-0.222343,-0.935277,0.202082
3,2025-09-30,-0.095400,-0.777550,0.361303
4,2025-08-31,-0.070545,-0.685187,0.426225
